In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Utility Functions for Testing

This notebook contains helper functions to test the different search algorithms.

### Dependencies

In [ ]:
import chess
import pprint
import numpy as np
import unittest

import import_ipynb
import Game
from Globals import Globals
from ChessAlgorithm import ChessAlgorithm

#### test_equals
Checks if two values are equal, then prints the results.

###### __<u>Arguments</u>__
``name (str):``  
The name of the test.

``value1 (any):``  
The first value that should be tested for equality.

``value2 (any):``  
The second value that should be tested for equality.

In [ ]:
def test_equals(name: str, value1: any, value2: any) -> None:
    test_passed = (value1 == value2)
    passed_string = 'PASSED' if test_passed else 'FAILED'
    print(f'{name} {passed_string}: {value1} = {value2}')

#### perform_chess_problem_tests
Runs a search algorithm against all available chess problems in order to test the functionality and effeciveness of this algorithm.

###### __<u>Arguments</u>__
``algorithm (type[ChessAlgorithm]):``  
The class of the chess search algorithm that white should use in the tests.

``max_depth (int, default: None):``  
The maximum search depth that the algorithm can handle. Chess problems whose solutions require more plies than this search depth are skipped. If set to `None`, no chess problems are skipped.  

###### __<u>Side effects</u>__
- The current visual output is overwritten.
- Test playthrough information is tracked and printed.

In [ ]:
def perform_chess_problem_tests(algorithm, max_depth: int=None) -> None: # TODO type
    test_game = Game.Game(algorithm, algorithm, endgame_tablebase_dir=None)

    for c in Globals.CHESS_PROBLEMS:
        test_game.play(chess_problem=c, search_depth_auto=True,
                max_depth=max_depth, automation=True)

    pprint.pprint(test_game.problem_playthroughs)
    print(f'\n{test_game.wins}')

#### perform_full_game_test
This is a full game test, in which a game of chess is played by two AIs. The game is defined by the arguments below. Information about the move times is calculated and displayed.

###### __<u>Arguments</u>__
``algorithm_white (type[ChessAlgorithm]):``  
The class of the chess algorithm that white should use to make a new move.

``algo_name_white (str):``  
The name of the algorithm that the white AI uses.  

``algorithm_black (type[ChessAlgorithm]):``  
The class of the chess algorithm that black should use to make a new move.

``algo_name_black (str):``  
The name of the algorithm that the black AI uses.  

``search_depth_white (int, default: 4):``  
The search depth for the white AI's search algorithm. 

``search_depth_black (int, default: 4):``  
The search depth for the black AI's search algorithm. 

###### __<u>Side effects</u>__
- The current visual output is overwritten.
- Test information is tracked and printed.

###### __<u>Returns _(Game.Game)_</u>__
The `Game` object of the game of chess that has been played.

In [ ]:
def perform_full_game_test(
    algorithm_white: type[ChessAlgorithm],
    algo_name_white: str,
    algorithm_black: type[ChessAlgorithm],
    algo_name_black: str,
    search_depth_white: int=4,
    search_depth_black: int=4
) -> Game.Game:

    game = Game.Game(algorithm_white, algorithm_black, search_depth_white, search_depth_black)
    game.play()

    white_times_mean = np.mean(game.move_times[chess.WHITE])
    white_times_std = np.std(game.move_times[chess.WHITE])
    black_times_mean = np.mean(game.move_times[chess.BLACK])
    black_times_std = np.std(game.move_times[chess.BLACK])
    print(f'{algo_name_white} (white) move times. Mean: {white_times_mean}, ' \
            + f'std. dev: {white_times_std}')
    print(f'{algo_name_black} (black) move times. Mean: {black_times_mean}, ' \
            + f'std. dev: {black_times_std}')

#### perform_fen_tests
Makes a move for two search algorithms on a board with a given FEN code, then returns the results. Performs this test for all FEN codes listed under `Globals.TEST_FEN_CODES`. This can be used to check if two algorithms make the same move in the same position, or if they choose a move with the same score.

###### __<u>Arguments</u>__
``first_algorithm (type[ChessAlgorithm]):``  
The class of the first chess algorithm that should be tested.

``second_algorithm (type[ChessAlgorithm]):``  
The class of the second chess algorithm that should be tested.

``only_scores (bool, default: False):``  
Whether or not to only compare the move scores. If `False`, both the move and the move scores of both algorithms need to match.

In [ ]:
def perform_fen_tests(
    first_algorithm: type[ChessAlgorithm],
    second_algorithm: type[ChessAlgorithm],
    only_scores=False
) -> None:
    
    for i, fen in enumerate(Globals.TEST_FEN_CODES):
        result = []
        for algorithm in ( first_algorithm, second_algorithm ):
            fen_test_game = Game.Game(algorithm, algorithm, opening_book=None,
                    endgame_tablebase_dir=None, fen=fen)
            color = fen_test_game.board.turn
            score, move, _, _ = fen_test_game.algorithm[color].make_move()
            result.append(score if only_scores else (move, score))
        test_equals(f'{i}. Test code {fen}', result[0], result[1])

#### make_single_move
A simple test where a search algorithm makes a single move on a board defined by a FEN code. The board state after this move is displayed.

###### __<u>Arguments</u>__
``algorithm (type[ChessAlgorithm]):``  
The class of the chess algorithm that should be tested.

``search_depth (int):``  
The maximum search depth of the search algorithm.

``fen (str):``  
The FEN code representing the board state that the algorithm should make a move on.

In [ ]:
def make_single_move(algorithm: type[ChessAlgorithm], search_depth: int, fen: str) -> None:
    game = Game.Game(
        algorithm_white=algorithm,
        search_depth_white=search_depth,
        algorithm_black=algorithm,
        search_depth_black=search_depth,
        opening_book=None,
        endgame_tablebase_dir=None,
        fen=fen
    )
    color = game.board.turn
    score, move, _, move_list = game.algorithm[color].make_move()
    game.display(score=score, move=move, move_list=move_list)